подключаем библиотеки

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re

Основной список цветов, который был составлен вручную, в процессе парсинга данный список будет пополняться для распознования цветов в строке названия, при нестандартном имени

In [4]:
colors = ['Mirror Grey', 'Prism Crush Black', 'Nebula Blue', 'Midnight Black', 'Sky Blue',
             'Metallic Gray', 'Dark Gray', 'Phantom Blue', 'Phantom Black', 'Coral White', 'Sapphire Blue',
             'Product Red', 'McLaren Edition', 'Space Grey',
             'Frozen Silver', 'Iceberg Grey', 'Aura Glow', 'Ice Silver'
             'Rose Gold', 'Black', 'Blue', 'White', 'Red',
             'Grey', 'Gold', 'Green', 'Pink', 'Purple', 'Brown', 'Silver', 'Black Fashion Blue', 'Metallic Blue']

In [4]:
chromedriver = './chromedriver.exe'

In [14]:
options = webdriver.ChromeOptions()
options.add_argument('headless')  # для открытия headless-браузера
browser = webdriver.Chrome(executable_path=chromedriver)

Схема датафрейма

In [96]:
df = pd.DataFrame({'name': [],
                   'brand': [],
                   'year': [],
                   'cost': [],
                   'discount': [],
                   'color': [],
                   'memory_size_Gb': [],
                   'dual_sim': [],
                   'discounted_goods': [],
                  })
df

,name,brand,year,cost,discount,color,memory_size_Gb,dual_sim,discounted_goods


Сайт, который парсим. next_page это страница с телефонами

In [302]:
# Переход на страницу входа
home_page = 'https://quke.ru/shop/smartfony'
next_page = 'https://quke.ru/shop/smartfony?order=desc-popular&\
pfrom=&pto=&c%5B152%5D%5Bfrom%5D=&c%5B152%5D%5Bto%5D=&c%5B155%5D%5Bfrom\
%5D=&c%5B155%5D%5Bto%5D=&c%5B154%5D%5Bfrom%5D=&c%5B154%5D%5Bto%5D=\
&c%5B144%5D%5Bfrom%5D=&c%5B144%5D%5Bto%5D=&c%5B157%5D%5Bfrom%5D=\
&c%5B157%5D%5Bto%5D=&c%5B160%5D%5Bfrom%5D=&c%5B160%5D%5Bto%5D=\
&c%5B176%5D%5Bfrom%5D=&c%5B176%5D%5Bto%5D=&c%5B170%5D%5Bfrom%5D=\
&c%5B170%5D%5Bto%5D=&page='

Функция для парсинга года телефона из описания

In [6]:
def parse_year(full_name):
    year = re.search(r'\([0-9]{4}\)', full_name)
    if year:
        full_name = full_name.replace(str(year[0]) + ' ', '')
        year = int(year[0][1:-1])
    else:
        year = None
    return full_name, year

Очистка описания от ненужных обозначений

In [ ]:
def clear_full_name(full_name):
    full_name = full_name.replace('EU ', '')
    full_name = re.sub(r'\([А-Яа-я]*\)', '', full_name)
    full_name = re.sub(r'\([A-Za-z]*\)| \([A-Za-z0-9\s]*\)', 'ok', full_name)
    full_name = re.sub(r'LTE ', '', full_name)
    return full_name

Функция парсинга, телефон с двумя симками

In [ ]:
def parse_dual_sim(full_name):
    if full_name.find('Dual sim') != -1 or full_name.find('Dual Sim') != -1:
        dual_sim = True
        full_name = full_name.replace('Dual sim ', '')
        full_name = full_name.replace('Dual Sim ', '')
    else:
        dual_sim = False
    return full_name, dual_sim

Парсинг уцененый ли товар

In [9]:
def parse_discounted_goods(full_name):
    if full_name.find('Уцененный товар ') != -1:
        discounted_goods = True
        full_name = full_name.replace('Уцененный товар ', '')
    else:
        discounted_goods = False
    return full_name, discounted_goods

Парсинг цвета телефона и добавление нового в массив color

In [10]:
def parse_color(full_name, colors):
    check_to_memory = '\s[0-9]{2,3}[gb|Gb]{2}\s([\sA-Za-z]+$)'
    color = re.search(check_to_memory, full_name)
    if color:
        color = color.group(1)
        pattern = "\s" + color + "$"
        full_name = re.sub(pattern, '', full_name)
    else:
        colors = sorted(colors, key=len, reverse = True)
        for colr in colors:
            if full_name.replace(' ', '').find(str(colr.replace(' ', ''))) != -1:
                color = str(colr)
                pattern = "\s" + color
                full_name = re.sub(pattern, '', full_name)
                break
            else:
                color = None
    return full_name, color

Парсинг размера встроенной памяти телефона

In [11]:
def parse_memory(full_name):
    pattern = "\d{2,3}[Gb|gb]{2}"
    memory = re.search(pattern, full_name)
    if memory is not None:
        memory = int(memory[0][:-2])
        full_name = re.sub("\s" + pattern, '', full_name)
    return full_name, memory

Не использовать ячейку. возможность перепроверки нераспарсинных телефонов

In [12]:
def parse_err(each_div, df, colors, error_list):
    cost = int(each_div.get('data-price'))

    full_name = each_div.find('a', {'class':'b-card__img-link'}).get('title')
    print(full_name)

    full_name, year = parse_year(full_name)
    full_name = clear_full_name(full_name)
    print("full " + full_name)
    full_name, dual_sim = parse_dual_sim(full_name)
    full_name, discounted_goods = parse_discounted_goods(full_name)

    #print (cost + ' name ' + name.rsplit(' ',1)[0] + ' memory ' + name.replace('EU ', '').rsplit(' ', 2)[1] + ' brand ' + name.split(' ', 1)[0] + ' color ' + name.rsplit(' ',1)[1])
    #print(full_name.rsplit(' ',1)[0])
    full_name, color = parse_color(full_name, colors)
    print(color)
    if color:
        colors.append(color)
    else:
        return df, colors, error_list

    full_name, memory = parse_memory(full_name)       


    brand = full_name.split(' ', 1)[0]

    print("name " + full_name)
    print("color " + color)
    print("brand " + brand)
    print("year " + str(year))
    print("memory " + str(memory) + '\n')
    df = df.append({'name': full_name,
                    'brand': brand,
                    'year': year,
                    'cost': cost,
                    'color': color,
                    'memory_size_Gb': memory,
                    'dual_sim': dual_sim,
                    'discounted_goods': discounted_goods}, ignore_index=True)
    return df, colors, error_list

основная функция функция парсинга

In [87]:
def parse_soup(soup, df, colors, error_list, find_err):
    for id, each_div in enumerate(soup.findAll('div',{'class':'b-product-list__col'})):
        cost = int(each_div.get('data-price'))
        try:
            discount = each_div.find('span',{'class':'b-card__discount-value'}).get_text()
            discount = int(discount.replace(' ', '').replace('-', ''))
        except:
            discount = 0

        full_name = each_div.find('a', {'class':'b-card__img-link'}).get('title')
        print(full_name)
        
        full_name, year = parse_year(full_name)
        full_name = clear_full_name(full_name)
        print("full " + full_name)
        full_name, dual_sim = parse_dual_sim(full_name)
        full_name, discounted_goods = parse_discounted_goods(full_name)

        full_name, color = parse_color(full_name, colors)
        
        if color:
            colors.append(color)
        else:
            if not find_err:
                error_list.append(each_div)
                continue
        
        full_name, memory = parse_memory(full_name)       

            
        brand = full_name.split(' ', 1)[0]
        
        print("name " + full_name)
        print("color " + color)
        print("brand " + brand)
        print("year " + str(year))
        print("memory " + str(memory))
        print("discount:" + str(discount) + '\n')


        df = df.append({'name': full_name,
                        'brand': brand,
                        'year': year,
                        'cost': cost,
                        'discount': discount,
                        'color': color,
                        'memory_size_Gb': memory,
                        'dual_sim': dual_sim,
                        'discounted_goods': discounted_goods}, ignore_index=True)
    return df, colors, error_list

Парсим сайт, сохраняя каждую страницу в массив

In [704]:
id = 0
error_list = []
allhtml = []
while(id != 341):
    try:
        browser.get(next_page + str(id))
    except ValueError:
        print("End of parsing")
        break
    else:
        allhtml.append(browser.page_source)
    print(id)
    id += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

Сохраняем каждую страницу сайта в файл, чтобы не нагружать сайт и увеличивая скорость локального парсинга

In [740]:
for j in range(341):
    file_1 = open("./html/file" + str(j) + ".html", "w", encoding='utf-8')
    file_1.write(allhtml[j])  
    file_1.close()

ПАРСИНГ САЙТА

In [97]:
erl = []
for j in range(1, 340):
    file_1 = open("./html/file" + str(j) + ".html", "r", encoding='utf-8')
    df, colors, error_list = parse_soup(BeautifulSoup(file_1.read()), df, colors, erl)

Samsung Galaxy A50 (2019) 64Gb Black
full Samsung Galaxy A50 64Gb Black
name Samsung Galaxy A50
color Black
brand Samsung
year 2019
memory 64
discount:2640

Samsung Galaxy A50 (2019) 64Gb Blue
full Samsung Galaxy A50 64Gb Blue
name Samsung Galaxy A50
color Blue
brand Samsung
year 2019
memory 64
discount:2000

Samsung Galaxy A50 (2019) 64Gb White
full Samsung Galaxy A50 64Gb White
name Samsung Galaxy A50
color White
brand Samsung
year 2019
memory 64
discount:2840

Samsung Galaxy A10 (2019) 32Gb Black
full Samsung Galaxy A10 32Gb Black
name Samsung Galaxy A10
color Black
brand Samsung
year 2019
memory 32
discount:2340

Samsung Galaxy A40 (2019) 64Gb Black
full Samsung Galaxy A40 64Gb Black
name Samsung Galaxy A40
color Black
brand Samsung
year 2019
memory 64
discount:3040

Samsung Galaxy A10 (2019) 32Gb Blue
full Samsung Galaxy A10 32Gb Blue
name Samsung Galaxy A10
color Blue
brand Samsung
year 2019
memory 32
discount:2340

Samsung Galaxy A10 (2019) 32Gb Red
full Samsung Galaxy A10 32Gb 

Xiaomi Redmi Note 8 3 32Gb EU White
full Xiaomi Redmi Note 8 3 32Gb White
name Xiaomi Redmi Note 8 3
color White
brand Xiaomi
year None
memory 32
discount:0

Xiaomi Redmi 7A 2 16Gb EU Blue
full Xiaomi Redmi 7A 2 16Gb Blue
name Xiaomi Redmi 7A 2
color Blue
brand Xiaomi
year None
memory 16
discount:840

Huawei Nova 5T 6 128Gb Blue
full Huawei Nova 5T 6 128Gb Blue
name Huawei Nova 5T 6
color Blue
brand Huawei
year None
memory 128
discount:5000

Samsung Galaxy A50S 6 128Gb White
full Samsung Galaxy A50S 6 128Gb White
name Samsung Galaxy A50S 6
color White
brand Samsung
year None
memory 128
discount:0

Xiaomi Redmi Note 8 4 128Gb EU White
full Xiaomi Redmi Note 8 4 128Gb White
name Xiaomi Redmi Note 8 4
color White
brand Xiaomi
year None
memory 128
discount:2900

OnePlus 7T Pro 8 256Gb Nebula Blue
full OnePlus 7T Pro 8 256Gb Nebula Blue
name OnePlus 7T Pro 8
color Nebula Blue
brand OnePlus
year None
memory 256
discount:9000

Xiaomi Redmi 8 3 32Gb EU Black
full Xiaomi Redmi 8 3 32Gb Black
na

Nokia 2.2 2 16 Gb Silver
full Nokia 2.2 2 16 Gb Silver
name Nokia 2.2 2 16 Gb
color Silver
brand Nokia
year None
memory None
discount:0

Huawei Honor 10i 128Gb Blue
full Huawei Honor 10i 128Gb Blue
name Huawei Honor 10i
color Blue
brand Huawei
year None
memory 128
discount:2200

Huawei Honor 10 Lite 3 64Gb Sapphire Blue
full Huawei Honor 10 Lite 3 64Gb Sapphire Blue
name Huawei Honor 10 Lite 3
color Sapphire Blue
brand Huawei
year None
memory 64
discount:2200

Meizu 15 Lite 4 64Gb EU Red
full Meizu 15 Lite 4 64Gb Red
name Meizu 15 Lite 4
color Red
brand Meizu
year None
memory 64
discount:0

Huawei Honor 7A Blue
full Huawei Honor 7A Blue
name Huawei Honor 7A
color Blue
brand Huawei
year None
memory None
discount:1240

Xiaomi Mi A3 4 128Gb EU White
full Xiaomi Mi A3 4 128Gb White
name Xiaomi Mi A3 4
color White
brand Xiaomi
year None
memory 128
discount:2200

Samsung Galaxy J2 Core SM-J260 Black
full Samsung Galaxy J2 Core SM-J260 Black
name Samsung Galaxy J2 Core SM-J260
color Black
bra

Lenovo S5 64Gb EU Black
full Lenovo S5 64Gb Black
name Lenovo S5
color Black
brand Lenovo
year None
memory 64
discount:1500

Realme 5 3 64Gb Blue
full Realme 5 3 64Gb Blue
name Realme 5 3
color Blue
brand Realme
year None
memory 64
discount:0

Meizu Note 8 4 64Gb EU Purple
full Meizu Note 8 4 64Gb Purple
name Meizu Note 8 4
color Purple
brand Meizu
year None
memory 64
discount:0

Caterpillar S60 Black
full Caterpillar S60 Black
name Caterpillar S60
color Black
brand Caterpillar
year None
memory None
discount:0

Nokia 2.3 32Gb Dual Sim Green
full Nokia 2.3 32Gb Dual Sim Green
name Nokia 2.3
color Green
brand Nokia
year None
memory 32
discount:0

Wigor V3 Black
full Wigor V3 Black
name Wigor V3
color Black
brand Wigor
year None
memory None
discount:0

Samsung Galaxy A51 64Gb Red
full Samsung Galaxy A51 64Gb Red
name Samsung Galaxy A51
color Red
brand Samsung
year None
memory 64
discount:0

Itel it5630 Grey
full Itel it5630 Grey
name Itel it5630
color Grey
brand Itel
year None
memory None

Philips S260 Black
full Philips S260 Black
name Philips S260
color Black
brand Philips
year None
memory None
discount:0

Meizu Note 8 4 64Gb EU Red
full Meizu Note 8 4 64Gb Red
name Meizu Note 8 4
color Red
brand Meizu
year None
memory 64
discount:3700

Alcatel 1S 5024D (2019) Metallic Black
full Alcatel 1S 5024D Metallic Black
name Alcatel 1S 5024D Metallic
color Black
brand Alcatel
year 2019
memory None
discount:0

Tecno Pop 2S Aurora Purple
full Tecno Pop 2S Aurora Purple
name Tecno Pop 2S Aurora
color Purple
brand Tecno
year None
memory None
discount:0

Philips Xenium E125 Blue
full Philips Xenium E125 Blue
name Philips Xenium E125
color Blue
brand Philips
year None
memory None
discount:0

Apple iPhone 7 Plus 32Gb Gold
full Apple iPhone 7 Plus 32Gb Gold
name Apple iPhone 7 Plus
color Gold
brand Apple
year None
memory 32
discount:0

Alcatel 3025X Metallic Gray
full Alcatel 3025X Metallic Gray
name Alcatel 3025X
color Metallic Gray
brand Alcatel
year None
memory None
discount:0

Meiz

...

Samsung Galaxy J5 (2016)
full Samsung Galaxy J5ok
Samsung Galaxy A9 Pro
full Samsung Galaxy A9 Pro
Samsung Galaxy S7 Mini
full Samsung Galaxy S7 Mini
Samsung SM-B360E
full Samsung SM-B360E
Samsung Galaxy J5
full Samsung Galaxy J5
Samsung Galaxy Mega On SM-G600
full Samsung Galaxy Mega On SM-G600
Samsung Galaxy Grand On
full Samsung Galaxy Grand On
Samsung Galaxy J3
full Samsung Galaxy J3



321



len=322
Samsung Galaxy S6 Active
full Samsung Galaxy S6 Active
Samsung SM-430
full Samsung SM-430
Samsung Galaxy E5 SM-E500HD Brown
full Samsung Galaxy E5 SM-E500HD Brown
name Samsung Galaxy E5 SM-E500HD
color Brown
brand Samsung
year None
memory None
discount:4200

Samsung Galaxy S5 Prime
full Samsung Galaxy S5 Prime
Samsung Galaxy F
full Samsung Galaxy F
Samsung Galaxy S4 GT-I9505 16Gb Brown
full Samsung Galaxy S4 GT-I9505 16Gb Brown
name Samsung Galaxy S4 GT-I9505
color Brown
brand Samsung
year None
memory 16
discount:6000

Samsung Galaxy S5 16Gb Crystal Edition
full Samsung Galaxy S5 16Gb

Meizu 16th 8 128Gb M882H Midnight Black
full Meizu 16th 8 128Gb M882H Midnight Black
name Meizu 16th 8 M882H
color Midnight Black
brand Meizu
year None
memory 128
discount:0

Meizu M3 Max 64Gb S685H Silver
full Meizu M3 Max 64Gb S685H Silver
name Meizu M3 Max S685H
color Silver
brand Meizu
year None
memory 64
discount:0

Xiaomi Redmi S1
full Xiaomi Redmi S1
name Xiaomimi S1
color Red
brand Xiaomimi
year None
memory None
discount:0

Xiaomi Redmi S3
full Xiaomi Redmi S3
name Xiaomimi S3
color Red
brand Xiaomimi
year None
memory None
discount:0

Xiaomi Valentino
full Xiaomi Valentino
Xiaomi X1
full Xiaomi X1
Xiaomi Redmi Note 5X
full Xiaomi Redmi Note 5X
name Xiaomimi Note 5X
color Red
brand Xiaomimi
year None
memory None
discount:0

Xiaomi Redmi Note 3 Prime
full Xiaomi Redmi Note 3 Prime
name Xiaomimi Note 3 Prime
color Red
brand Xiaomimi
year None
memory None
discount:0

Xiaomi Redmi Note 4X 32Gb+3Gb Hatsune Miku Special Edition
full Xiaomi Redmi Note 4X 32Gb+3Gb Hatsune Miku Special E

Alcatel 4034D Pixi 4 (4.0) Black Blue
full Alcatel 4034D Pixi 4 (4.0) Black Blue
name Alcatel 4034D Pixi 4 (4.0) Blue
color Black
brand Alcatel
year None
memory None
discount:0

Alcatel 2008G White
full Alcatel 2008G White
name Alcatel 2008G White
color White 
brand Alcatel
year None
memory None
discount:0

Alcatel 2051D Metal Silver
full Alcatel 2051D Metal Silver
name Alcatel 2051D Metal
color Silver
brand Alcatel
year None
memory None
discount:0

Alcatel 2051D Pure White
full Alcatel 2051D Pure White
name Alcatel 2051D Pure White
color White 
brand Alcatel
year None
memory None
discount:0

Alcatel Shine Lite White
full Alcatel Shine Lite White
name Alcatel Shine Lite White
color White 
brand Alcatel
year None
memory None
discount:0

Alcatel One Touch 1020D White
full Alcatel One Touch 1020D White
name Alcatel One Touch 1020D White
color White 
brand Alcatel
year None
memory None
discount:0

Alcatel POP4 7070X Slate Black
full Alcatel POP4 7070X Slate Black
name Alcatel POP4 7070X Sl

Alcatel 1013D White
full Alcatel 1013D White
name Alcatel 1013D White
color White 
brand Alcatel
year None
memory None
discount:0




333



len=424
Alcatel POP 2 (4) 4045D Pure White
full Alcatel POP 2ok 4045D Pure White
name Alcatel POP 2ok 4045D Pure White
color White 
brand Alcatel
year None
memory None
discount:0

Alcatel POP 2 (4) 4045D Volcano Black
full Alcatel POP 2ok 4045D Volcano Black
name Alcatel POP 2ok 4045D Volcano
color Black
brand Alcatel
year None
memory None
discount:0

Alcatel PIXI 3(4) 4013D White
full Alcatel PIXI 3(4) 4013D White
name Alcatel PIXI 3(4) 4013D White
color White 
brand Alcatel
year None
memory None
discount:0

Alcatel Pop D3 4035D Black Dark Aubergine
full Alcatel Pop D3 4035D Black Dark Aubergine
name Alcatel Pop D3 4035D Dark Aubergine
color Black
brand Alcatel
year None
memory None
discount:0

Alcatel Pop D3 4035D White Dark Aubergine
full Alcatel Pop D3 4035D White Dark Aubergine
name Alcatel Pop D3 4035DDark Aubergine
color White 
brand Alcate




336



len=448
Alcatel One Touch 2005D Coralline
full Alcatel One Touch 2005D Coralline
name Alcatel One Touch 2005Dline
color Coral
brand Alcatel
year None
memory None
discount:0

Alcatel One Touch 292 Pure White
full Alcatel One Touch 292 Pure White
name Alcatel One Touch 292 Pure White
color White 
brand Alcatel
year None
memory None
discount:0

Alcatel One Touch 990 Chrome
full Alcatel One Touch 990 Chrome
Alcatel One Touch 985D Raven Black
full Alcatel One Touch 985D Raven Black
name Alcatel One Touch 985D Raven
color Black
brand Alcatel
year None
memory None
discount:0

Alcatel One Touch 720D Pure White
full Alcatel One Touch 720D Pure White
name Alcatel One Touch 720D Pure White
color White 
brand Alcatel
year None
memory None
discount:0

Alcatel One Touch 720D Chrome White
full Alcatel One Touch 720D Chrome White
name Alcatel One Touch 720D Chrome White
color White 
brand Alcatel
year None
memory None
discount:0

Alcatel One Touch 720D Lilac
full Alcatel One Touch 720D Lilac

Сохраняем датафрейм в файл

In [107]:
df['cost'] = df['cost'].astype(int)
df['discount'] = df['discount'].astype(int)

df.to_csv('./dataframe.csv', index=False)

неиспользуемая функция, парсинга описания с ошибками

In [111]:
for err_parse in erl:
    df, colors, error_list = parse_err(err_parse, df, colors, erl)
    print("len=" + str(len(erl)))

Tecno Camon 12 Dark Jade
full Tecno Camon 12 Dark Jade
None
len=493
Tecno Camon 12 Sky Cyan
full Tecno Camon 12 Sky Cyan
None
len=493
Nokia 230 Dual Sim Dark silver
full Nokia 230 Dual Sim Dark silver
None
len=493
Уцененный товар Samsung Galaxy A50 (2019) 64Gb Синий
full Уцененный товар Samsung Galaxy A50 64Gb Синий
None
len=493
Vertex Impress Stone Grafit
full Vertex Impress Stone Grafit
None
len=493
Vertex Impress Click Nfc Grafit
full Vertex Impress Click Nfc Grafit
None
len=493
Уцененный товар Xiaomi Mi8 6 64Gb EU Черный
full Уцененный товар Xiaomi Mi8 6 64Gb Черный
None
len=493
Tecno Spark 4 Energetic Orange
full Tecno Spark 4 Energetic Orange
None
len=493
Уцененный товар AirPods 2 (без беспроводной зарядки чехла) Белые
full Уцененный товар AirPods 2 (без беспроводной зарядки чехла) Белые
None
len=493
Joy's S6 Orange
full Joy's S6 Orange
None
len=493
Joy's S9 Chocolate
full Joy's S9 Chocolate
None
len=493
Joy's S11 Champaign
full Joy's S11 Champaign
None
len=493
Уцененный товар Ap

full Samsung Galaxy S5 Prime
None
len=493
Samsung Galaxy F
full Samsung Galaxy F
None
len=493
Samsung Galaxy Round
full Samsung Galaxy Round
None
len=493
Samsung Galaxy Ace II x GT-S7560M
full Samsung Galaxy Ace II x GT-S7560M
None
len=493
Samsung S5360 Galaxy Y Hello Kitty
full Samsung S5360 Galaxy Y Hello Kitty
None
len=493
Samsung I9502 Galaxy S IV
full Samsung I9502 Galaxy S IV
None
len=493
Samsung Tizen
full Samsung Tizen
None
len=493
Samsung Galaxy Chat B5330
full Samsung Galaxy Chat B5330
None
len=493
Samsung i9100 Galaxy S II
full Samsung i9100 Galaxy S II
None
len=493
Samsung C3752
full Samsung C3752
None
len=493
Samsung Galaxy Mini 2 S6500 (Jena)
full Samsung Galaxy Mini 2 S6500ok
None
len=493
Samsung C3520 La Fleur
full Samsung C3520 La Fleur
None
len=493
Samsung S5610 Metallic
full Samsung S5610 Metallic
None
len=493
Samsung C3300 Hello Kitty
full Samsung C3300 Hello Kitty
None
len=493
Huawei Y6 Pro (2017)
full Huawei Y6 Prook
None
len=493
Huawei Y6 Pro (2017)
full Huawei Y

In [101]:
df.describe()

,cost,discount,dual_sim,discounted_goods
count,6305.000000,6305.000000,6305.000000,6305.000000
mean,10486.285646,1101.260587,0.024742,0.033941
std,10880.046341,2585.897871,0.155351,0.181093
min,0.000000,0.000000,0.000000,0.000000
25%,3135.000000,0.000000,0.000000,0.000000
50%,7790.000000,0.000000,0.000000,0.000000
75%,13990.000000,850.000000,0.000000,0.000000
max,99990.000000,31400.000000,1.000000,1.000000


In [40]:
df.isnull().sum()

name                   0
brand                  0
year                6059
cost                   0
color                  0
memory_size_Gb      3550
dual_sim               0
discounted_goods       0
dtype: int64

In [108]:
df.dtypes

name                 object
brand                object
year                 object
cost                  int64
discount              int64
color                object
memory_size_Gb       object
dual_sim            float64
discounted_goods    float64
dtype: object

In [102]:
df.count()

name                6305
brand               6305
year                 228
cost                6305
discount            6305
color               6305
memory_size_Gb      2737
dual_sim            6305
discounted_goods    6305
dtype: int64